In [ ]:
%load_ext line_profiler
import numpy as np

class TicTacToe():
    def __init__(self, ai_start=False):
        self.board = np.array(['']*9).reshape(3, 3)
        if ai_start:
            self.ai = 'X'
            self.user = 'O'
        else:
            self.ai = 'O'
            self.user = 'X'
        self.turn = 'X'
        self.win_dict = {'X': 1, 'O': -1}
        self.start(ai_start)

    def start(self, ai_start):
        print(f"User: {self.user} and AI: {self.ai}")
        if ai_start:
            self.ai_move()
            self.print_board()

    def interpret_result(self, res):
        if res == 0:
            print("It's a tie! Please start a new game")
        elif res == 1:
            print("X won! Please start a new game")
        else:
            print("O won! Please start a new game")

    def print_board(self):
        print(self.board)
    
    def legal_moves(self, print1=False):
        empty = []
        for idx, row in enumerate(self.board):
            for j, column in enumerate(row):
                if column == '':
                    empty.append((idx, j))
        return empty                    

    def game_over(self):
        if len(self.legal_moves()) == 0:
            # No more legal moves, return 0 for tie
            return 0
    
        # Check rows
        for row in self.board:
            row_item = [item for item in row if item != '']
            if len(row_item) == self.board.shape[0] and len(set(row_item)) == 1 and '' not in set(row_item):
                return self.win_dict[row_item[0]]

        # Check columns
        for index in range(len(self.board)):
            column_item = self.board[:,index]
            if len(column_item) ==  self.board.shape[1] and len(set(column_item)) == 1 and '' not in set(column_item):
                return self.win_dict[column_item[0]]
        
        # Check diag left-right
        left_diag = [self.board[i][i] for i in range(len(self.board))]
        if len(set(left_diag)) == 1 and '' not in set(left_diag):
            return self.win_dict[left_diag[0]]
        
        # Check diag right-left
        right_diag = [self.board[len(self.board)-1-i][i] for i in range(len(self.board)-1,-1,-1)]
        if len(set(right_diag)) == 1 and '' not in set(right_diag):
            return self.win_dict[right_diag[0]]
        return None   
        
    
    def user_play(self, input_x, input_y):
        if self.board[input_x][input_y] != '':
            print("That spot is taken! Please choose another coordinate")
            return
        
        # Do not allow play if the game is alread over
        if self.game_over():
            print("Game is already over! Please start a new game")
            self.print_board()
            return
        
        # Place user's coordinate on the board
        self.board[input_x][input_y] = self.user
        
        # Check if the user just won
        res = self.game_over()
        if res is not None:
            self.print_board()
            self.interpret_result(res)
            return
        
        # Allow ai to move
        self.ai_move()
        res = self.game_over()
        if res is not None:
            self.print_board()
            self.interpret_result(res)
            return

        # Print board after every turn
        self.print_board()
        
    def ai_move(self):
        move = None
        best_score = -float("inf")
        legal_moves = self.legal_moves()
        for legal_move in legal_moves:
            self.board[legal_move[0]][legal_move[1]] = self.ai
            score = self.minimax(0, False)
            self.board[legal_move[0]][legal_move[1]] = ''
            if score > best_score:
                best_score = score
                move = legal_move
        self.board[move[0]][move[1]] = self.ai
                
    def minimax(self, depth, is_maximizing):
        result = self.game_over()
        if result != None:
            return result*5

        legal_moves = self.legal_moves()
        if is_maximizing:
            best_score = -float("inf")
            for legal_move in legal_moves:
                self.board[legal_move[0]][legal_move[1]] = self.ai
                score = self.minimax(depth+1, False)
                self.board[legal_move[0]][legal_move[1]] = ''
                best_score = max(score, best_score)
            return best_score
        else:
            best_score = float("inf")
            for legal_move in legal_moves:
                self.board[legal_move[0]][legal_move[1]] = self.user
                score = self.minimax(depth+1, True)
                self.board[legal_move[0]][legal_move[1]] = ''
                best_score = min(score, best_score)
            return best_score
                

In [ ]:
%lprun -f TicTacToe.minimax a = TicTacToe(True)        

In [ ]:
%lprun -f TicTacToe.minimax a.user_play(0,1)